# CSCI 3155 Recitation 6
October 5, 2018

## Reviewing inference rules

In the past few weeks, you've encountered a variety of formal inference rules of the form

$$\begin{array}{c}
s \in \text{Domain}(\sigma)\\
\hline
\sigma \vdash \texttt{Ident(s)} \Downarrow \sigma(s) \\
\end{array}\ (\text{Ident})$$

It's worth pausing briefly and reviewing how to think of and utilize these inference rules at a high level, as it's easy to get lost in the technical details.

Say we've just started designing our own programming language and have defined the abstract syntax. We now want to take programs defined in our abstract syntax and determine certain useful properties about them, e.g., "all variables in this program are defined before they are used", "this program evaluates to this value", etc. In PL jargon, each of these statements is called a *judgment*. We define each type of judgment inductively via a collection of inference rules -- that is, judgments of the form "all variables in this program are defined before they are used" have a collection of inference rules, judgments of the form "this program evaluates to this value" have their own separate collection of rules, etc. 
As programmer language designers, we can view a judgement's inference rules as instructions for how to implement a tool that automatically checks that the judgment holds.

Let's make this discussion concrete by reviewing the judgments we've encountered so far for Lettuce. We start with the abstract syntax.

In [1]:
sealed trait Expr

// Atoms
case class Const(v: Double) extends Expr
case object True extends Expr
case object False extends Expr
case class Ident(s: String) extends Expr

// Arithmetic Expressions
case class Plus(e1: Expr, e2: Expr) extends Expr
case class Minus(e1: Expr, e2: Expr) extends Expr
case class Mult(e1: Expr, e2: Expr) extends Expr
case class Div(e1: Expr, e2: Expr) extends Expr
case class Log(e: Expr) extends Expr 
case class Exp(e: Expr) extends Expr
case class Sine(e: Expr) extends Expr
case class Cosine(e: Expr) extends Expr

// Boolean Expressions
case class Geq(e1: Expr, e2:Expr) extends Expr
case class Eq(e1: Expr, e2: Expr) extends Expr
case class And(e1: Expr, e2: Expr) extends Expr
case class Or(e1: Expr, e2: Expr) extends Expr
case class Not(e: Expr) extends Expr

//If then else
case class IfThenElse(e: Expr, eIf: Expr, eElse: Expr) extends Expr

//Let bindings
case class Let(s: String, defExpr: Expr, bodyExpr: Expr) extends Expr

defined trait Expr
defined class Const
defined object True
defined object False
defined class Ident
defined class Plus
defined class Minus
defined class Mult
defined class Div
defined class Log
defined class Exp
defined class Sine
defined class Cosine
defined class Geq
defined class Eq
defined class And
defined class Or
defined class Not
defined class IfThenElse
defined class Let

The first judgment you encountered stated that variables in a Lettuce program are defined before they are used. The inference rules tell us precisely how to implement a tool that checks well-formedness. As implementors, we read the rules backwards. For example, $\text{(let-rule)}$ tells us that, in order to check well-formedness of the expression `Let(x, e1, e2)` given the set $S$ of defined variables, we need to
1. confirm that `e1` is well-formed
2. confirm that `e2` is well-formed under defined variables $S \cup \{x\}$

$\fbox{$WellFormed(\texttt{e}, S)$}$

$$ \begin{array}{c}
\\
\hline
WellFormed(\texttt{Const(f)}, S) \\
\end{array} \text{(const-rule)} $$

$$\begin{array}{c}
x \in S \\
\hline
WellFormed(\texttt{Ident(x)}, S) \\
\end{array} \text{(ident-rule)} $$

$$ \begin{array}{c}
WellFormed(\texttt{e1}, S) \;\;\; WellFormed(\texttt{e2}, S)\;\;\; T \in \{ \texttt{Plus}, \texttt{Minus}, \texttt{Mult}, \texttt{Div}, \texttt{Geq}, \texttt{Eq}, \texttt{And}, \texttt{Or} \} \\
\hline
WellFormed(\texttt{T(e1, e2)}, S) \\
\end{array} \text{(well-formed-binary-op)} $$

$$ \begin{array}{c}
WellFormed(\texttt{e1}, S) \;\;\; T \in \{ \texttt{Log}, \texttt{Exp}, \texttt{Sine}, \texttt{Cosine}, \texttt{Not}, \texttt{Eq} \} \\
\hline
WellFormed(\texttt{T(e1)}, S) \\
\end{array} \text{(well-formed-unary-op)} $$

$$\begin{array}{c}
\color{red}{WellFormed(\texttt{e1}, S) \;\;\; WellFormed(\texttt{e2}, S \cup \{ x\} )} \\
\hline
\color{red}{WellFormed(\texttt{Let(x, e1, e2)}, S)} \\
\end{array} \text{(let-rule)} $$

In [2]:
def isWellFormed(e: Expr, S: Set[String]): Boolean = e match {
    /* ... */
    
    /**/
    case Let(x, e1, e2) => isWellFormed(e1, S) && { 
        val S1 = S + x 
        isWellFormed(e2, S1)
    }
    /**/

    /* ... */
}

defined function isWellFormed

Another example we've seen: the judgment form $eval(e, \sigma) = v$. We used this judgment form to implement an interpreter for Lettuce programs.

> Note on terminology: If a judgment and its inference rules concern the dynamic execution of the program (as opposed to, for example, the well-formedness check above), then they are often called the *operational semantics*.

Once again, we read the rules backwards to understand how to evaluate expressions of each type. For example, consider the $\text{(let-binding-ok)}$ rule. It tell us that, in order to evaluate the expression `Let(x,e1,e2)` under environment $\sigma$, we need to:
1. Evaluate `e1` to get the value $v_1$.
2. Evaluate `e2` under $\sigma$ extended so that `x` points to $v_1$, to get $v_2$.
3. Return $v_2$.

$\fbox{$eval(e, \sigma) = v$}$

$$\begin{array}{c}
\\
\hline
eval(\texttt{Const(v)}, \sigma) = v \\
\end{array} \text{(const-rule)} $$

$$\begin{array}{c}
x \in \text{domain}(\sigma) \\
\hline
eval(\texttt{Ident(x)}, \sigma) = \sigma(\texttt{x}) \\
\end{array} \text{(ident-ok-rule)}\ \;\;\; $$

$$\begin{array}{c}
eval(\texttt{e1}, \sigma) = v_1,\; \; eval(\texttt{e2}, \sigma) = v_2,\ \ v_1 \in \mathbb{R},\ \ v_2 \in \mathbb{R}, \; \; \texttt{T} \in \{ \texttt{Plus, Minus, Mult} \}  \\
\hline
eval(\texttt{T(e1, e2)}, \sigma) = f_T(v_1, v_2) \\
\end{array} \text{(arith-binop-ok-rule)}$$

$$\begin{array}{c}
eval(\texttt{e}, \sigma) = v,\;\; v \in \mathbb{R} \; \; \texttt{T} \in \{ \texttt{Exp, Sine, Cosine} \}  \\
\hline
eval(\texttt{T(e)}, \sigma) = f_T(v) \\
\end{array} \text{(arith-unop-ok-rule)}$$

$$\begin{array}{c} 
\\
\hline
eval(\texttt{True}, \sigma) = true \\
\end{array} \text{(true rule)} \;\;\;\;
\begin{array}{c} 
\\
\hline
eval(\texttt{False}, \sigma) = false \\
\end{array}\text{(false rule)}
$$

$$\begin{array}{c} 
eval(\texttt{e1}, \sigma) = false\\
\hline
eval(\texttt{And}(e1, e2), \sigma) = false\\
\end{array} \text{(and-arg-1-ok-rule)}$$

$$\begin{array}{c} 
eval(\texttt{e1}, \sigma) = true\;\; eval(\texttt{e2}, \sigma) = v_2,\ \;\; v_2 \in \mathbb{B}\\
\hline
eval(\texttt{And}(e1, e2), \sigma) = v_2\\
\end{array} \text{(and-arg-2-ok-rule)}$$

$$\begin{array}{c} 
\color{red}{eval(\texttt{e1}, \sigma) = v_1,}\ 
\color{red}{v_1 \not= \mathbf{error}}\;\; 
\color{red}{eval(\texttt{e2}, \sigma[x \mapsto v_1]) = v_2,}\ \;\;
\color{red}{v_2 \not= \mathbf{error}}\\
\hline
\color{red}{eval(\texttt{Let(x,e1, e2)}, \sigma) = v_2}\\
\end{array} \text{(let-binding-ok)} $$

$$\vdots$$

In [2]:
/* 1. Define the values */
sealed trait Value 
case class NumValue(d: Double) extends Value
case class BoolValue(b: Boolean) extends Value
case object ErrorValue extends Value

/*2. Operators on values */

def valueToNumber(v: Value): Double = v match {
    case NumValue(d) => d
    case _ => throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a number")
}

def valueToBoolean(v: Value): Boolean = v match {
    case BoolValue(b) => b
    case _ => throw new IllegalArgumentException(s"Error: Asking me to convert Value: $v to a boolean")
}

defined trait Value
defined class NumValue
defined class BoolValue
defined object ErrorValue
defined function valueToNumber
defined function valueToBoolean

In [3]:
def evalExpr(e: Expr, env: Map[String, Value]): Value =  {    
    e match {
        /* ... */
        
        /**/
        case Let(x, e1, e2) => {
            val v1 = evalExpr(e1, env)  // eval e1
            val env2 = env + (x -> v1) // create a new extended env
            evalExpr(e2, env2) // eval e2 under that.
        }
        /**/
        
        /* ... */
    }
}

defined function evalExpr

### Exercise: implement a simple typechecker

Let's introduce a new judgment form $typeOf(\texttt{e}) = \tau$ that says that the Lettuce expression `e` has the type $\tau$. For simplicity, we assume `e` has no variables or `Let` expressions.

We have started defining the inference rules and implementation below. Your task is to extend the inference rules to handle `IfThenElse` expressions (on paper) and extend the implementation accordingly.

$\fbox{$typeOf(e) = \tau$}$

$$\begin{array}{c}
\\
\hline
typeOf(\texttt{Const(v)}) = Num \\
\end{array} \text{(const-rule)} $$

$$\begin{array}{c} 
\\
\hline
typeOf(\texttt{True}) = Bool \\
\end{array} \text{(true-rule)} \;\;\;\;
\begin{array}{c} 
\\
\hline
typeOf(\texttt{False}) = Bool \\
\end{array}\text{(false-rule)}
$$

$$\begin{array}{c}
typeOf(\texttt{e1}) = Num\hspace{1.5em} typeOf(\texttt{e2}) = Num\hspace{1.5em} \texttt{T} \in \{ \texttt{Plus, Minus, Mult} \}  \\
\hline
typeOf(\texttt{T(e1, e2)}) = Num \\
\end{array} \text{(arith-binop-ok-rule)}$$

$$\begin{array}{c}
typeOf(\texttt{e1}) \neq Num\hspace{0.5em}\text{or}\hspace{0.5em}typeOf(\texttt{e2}) \neq Num\hspace{1.5em} \texttt{T} \in \{ \texttt{Plus, Minus, Mult} \}  \\
\hline
typeOf(\texttt{T(e1, e2)}) = TypeError \\
\end{array} \text{(arith-binop-nok-rule)}$$

$$\begin{array}{c}
typeOf(\texttt{e1}) = Bool\hspace{1.5em} typeOf(\texttt{e2}) = Bool\hspace{1.5em} \texttt{T} \in \{ \texttt{And, Or} \}  \\
\hline
typeOf(\texttt{T(e1, e2)}) = Bool \\
\end{array} \text{(bool-binop-ok-rule)}$$

$$\begin{array}{c}
typeOf(\texttt{e1}) \neq Bool\hspace{0.5em}\text{or}\hspace{0.5em}typeOf(\texttt{e2}) \neq Bool\hspace{1.5em} \texttt{T} \in \{ \texttt{And, Or} \}  \\
\hline
typeOf(\texttt{T(e1, e2)}) = TypeError \\
\end{array} \text{(bool-binop-nok-rule)}$$

$$\begin{array}{c}
typeOf(\texttt{e1}) = Bool\hspace{0.5em}\text{and}\hspace{0.5em}typeOf(\texttt{e2}) = typeOf(\texttt{e3})\hspace{0.5em} = \hspace{0.5em}\tau  \\
\hline
typeOf(\texttt{ifThenElse(e1, e2, e3)}) = \tau  \\
\end{array} \text{(ifthenelse-ok-rule)}$$

$$\begin{array}{c}
typeOf(\texttt{e1}) \neq Bool\hspace{0.5em}\text{or}\hspace{0.5em}typeOf(\texttt{e2}) \neq typeOf(\texttt{e3})\hspace{0.5em}\text{or}\hspace{0.5em} typeOf(\texttt{e2}) = TypeError \hspace{0.5em}\text{or}\hspace{0.5em} typeOf(\texttt{e3}) = TypeError = \hspace{0.5em}TypeError  \\
\hline
typeOf(\texttt{ifThenElse(e1, e2, e3)}) = TypeError  \\
\end{array} \text{(ifthenelse-nok-rule)}$$

In [11]:
def typeOf(e : Expr) : String = {
    
    def typeOf_arithBinop(e1 : Expr, e2 : Expr) = {
        if (typeOf(e1) == "Num" && typeOf(e2) == "Num") 
            "Num"
        else 
            "TypeError"
    }
    
    def typeOf_boolBinop(e1 : Expr, e2 : Expr) = {
        if (typeOf(e1) == "Bool" && typeOf(e2) == "Bool") 
            "Bool"
        else 
            "TypeError"
    }
    
    e match {
        // const-rule
        case Const(v) => "Num"
        // true-rule
        case True => "Bool"
        // false-rule
        case False => "Bool"

        // arith-binop-ok-rule + arith-binop-nok-rule
        case Plus(e1,e2) => typeOf_arithBinop(e1, e2)
        case Minus(e1,e2) => typeOf_arithBinop(e1, e2)
        case Mult(e1,e2) => typeOf_arithBinop(e1, e2)

        // bool-binop-ok-rule + bool-binop-nok-rule
        case And(e1, e2) => typeOf_boolBinop(e1, e2)
        case Or(e1, e2) => typeOf_boolBinop(e1, e2)
                
        case IfThenElse(e1, e2, e3) => {
            // YOUR CODE HERE
            if (typeOf(e1) == "Bool") {
                if (typeOf_arithBinop(e2, e3) == "Num"){
                    "Num"
                } else if (typeOf_boolBinop(e2, e3) == "Bool") {
                    "Bool"
                } else {
                    "TypeError"
                }
            } else {
                "TypeError"
            }
//             val t1 = typeOf(e1)
//             val t2 = typeOf(e2)
//             val t3 = typeOf(e3)
//             if (t1 == "Bool" && t2 == t3 && t2 != "TypeError") {
//                 t2
//             } else {
//                 "TypeError"
//             }
        }
    }
}

defined function typeOf

In [12]:
assert(typeOf(Const(1.0)) == "Num", "Test 1")
assert(typeOf(True) == "Bool", "Test 2")
assert(typeOf(Plus(Const(1.0), Const(2.0))) == "Num", "Test 3")
assert(typeOf(Plus(Const(1.0), True)) == "TypeError", "Test 4")

// Tests your addition
assert(typeOf(IfThenElse(True, Const(1.0), Const(2.0))) == "Num", "Test 5")
assert(typeOf(IfThenElse(Const(0.0), Const(1.0), Const(2.0))) == "TypeError", "Test 6")
assert(typeOf(IfThenElse(False, Const(1.0), True)) == "TypeError", "Test 7")

## Scoping
The scope of a defined variable is everywhere it can be **used** while still refering to that definition.

### Scala Scoping
In Scala, we have 4 major ways to *define* variables, each with it's own rules about the resulting scope for that variable. The 4 ways are through `var` / `val`, functions parameters, case clauses, and `def`.

In Scala, when a variable is defined it's scope is the current scoping block and every nested block which does not redefine the same name. A scoping block is any section of code which can contain variables that are then lost once the block ends. The following are examples of the "blocks" I am refering to:

* Function body:
```
def f() = SCOPING BLOCK
```
* Braces:
```
{
    SCOPING BLOCK
}
```
* Case
```
() match {
    case () => SCOPING BLOCK
}
```

When Scala goes through and tries to find out where the variable definition that corresponds to a variable use is, it looks for a definition in the current scope, then checks every "parent" scope until it finds the first one. If there would be a different definition of the same name had we kept going up to parent blocks, we say that the outer variable is "shadowed" by the inner variable.

In the following, I will refer to variables as `var1`, `var2`, etc. instead of variable names. Because we will talk about shadowing, variable names will be reused.

### `var` / `val`
These two language constructs have the same scoping rules, so we will just talk about `val`.

We declare 2 variables, both named `x`. The first is named `var1`, and is **shadowed** by the second, named `var2`. Though they have the same name, they are different variables, meaning they are stored at different locations. When we redefine `x`, we lose any way to refer to the old `x`. Since we don't ever redeclare `y`, we have access to it throughout the program.

In [13]:
{
    val x = 1   // var1
    /* BEGIN SCOPE OF `var1`*/
    val y = 'a' // var2
    /* BEGIN SCOPE OF `var2` */
    println(s"x is $x, y is $y")
    /* END SCOPE OF `var1` */
    {
        val x = 2 // var3
        /* BEGIN SCOPE OF `var3` */
        println(s"x is $x, y is $y") // y can still be accessed here!
        /* END SCOPE OF `var3` */
    }
    /* BEGIN SCOPE OF `var1` */
    println(s"x is $x, y is $y")
    /* END SCOPE OF `var1`, `var2` */
}

x is 1, y is a
x is 2, y is a
x is 1, y is a


x: Int = 1
y: Char = 'a'

### Function parameters
Function parameters are also variable declarations. They make variables available in the function body. They follow the same scoping rules as we saw above.

In [14]:
{
    def f(x: Int, y: Int): Unit = { // var1, var2
        /* BEGIN SCOPE OF `var1`, `var2` */
        println(s"x is $x, y is $y")
        /* END SCOPE OF `var1` */
        (
            (x: Char) => {
                /* BEGIN SCOPE OF `var3` */
                println(s"x is $x, y is $y")
                x
                /* END SCOPE OF `var3` */
            }
        )('3')
        /* BEGIN SCOPE OF `var1` */
        println(s"x is $x, y is $y")
        /* END SCOPE OF `var1`, `var2` */
    }
    f(1, 2)
}

x is 1, y is 2
x is 3, y is 2
x is 1, y is 2


defined function f

### Cases
Case clauses declare variables in their pattern, and make them available in the block after the arrow. They follow the same scoping rules as we saw above.

\* Note, the variables declared in a pattern are in scope within the guard as well.

In [15]:
{
    1 :: 2 :: Nil match {
        case x :: y :: _ => { // var1, var2
            /* BEGIN SCOPE OF `var1`, `var2` */
            println(s"x is $x, y is $y")
            3 match {
                /* END SCOPE OF `var1` */
                case x => { //var3
                    /* BEGIN SCOPE OF `var3` */
                    println(s"x is $x, y is $y")
                    x
                    /* END SCOPE OF `var3` */
                }
                /* BEGIN SCOPE OF `var1` */
            }
            println(s"x is $x, y is $y")
            /* END SCOPE OF `var1`, `var2` */
        }
    }
}

x is 1, y is 2
x is 3, y is 2
x is 1, y is 2


### `def`
`def` is what we use to define functions. It is similar to `val`, but it allows the defined variable to be used on the right hand side of the `=` as well. Note the difference between the below example and the `var` / `val` (the "BEGIN SCOPE" comes *right after* the `=`, instead of on the next line). This is what allows functions to be **recursive**

In [16]:
{
    def f(): Unit = /* BEGIN SCOPE OF `var1` */ {
        f()
    }
    f _ // This simple avoids applying the funtion, since it's an infinite loop
    /* END SCOPE OF `var1` */
}

defined function f
res15_1: () => Unit = <function0>

### Exercise: Scoping
For each variable use, say which line it's definition lies on. Then, list any variable definitions which have no use.

In [16]:
/*  1 */ {
/*  2 */     val x = 4
/*  3 */     val y = 5
/*  4 */     x match {
/*  5 */         case y => y
/*  6 */         case x => y
/*  7 */     }
/*  8 */     def f(x: Int, z: Int) {
/*  9 */         x match {
/* 10 */             case z => {
/* 11 */                 def f(y: Int) {
/* 12 */                     val z = 10
/* 13 */                     x + y + z
/* 14 */                 }
/* 15 */                 f(z)
/* 16 */             }
/* 17 */             case x => y
/* 18 */         }
/* 19 */     }
/* 20 */     {
/* 21 */         val x = 6
/* 22 */         val z = 7
/* 23 */         {
/* 24 */             x + y + z
/* 25 */         }
/* 26 */     }
/* 27 */ }

cmd16.sc:22: Int(7) does not take parameters
/* 23 */         {
                 ^

: 

x = 
y = 
z = 

### Exercise: Environments
Now, using our knowledge of scoping, lets walk through the execution of a program and list the environment at each step. Walk through the following program, and for each line, write the environment as follows:

For the program:
```
/* 1 */ val x = 10
/* 2 */ val y = 5
/* 3 */ def f() {
/* 4 */     val y = 6
/* 5 */ }
/* 6 */ f()
```
We would give the following answer:

| Line | Env |
|------|-----|
| 1 | `{}` |
| 2 | `{x: 10}` |
| 3 | `{x: 10, y: 5, f: func}` |
| 6 | `{x: 10, y: 5, f: func}` |
| 4 | `{x: 10, y: 6, f: func}` |

Give this table for the following code:

In [17]:
/*  1 */ val x = 3
/*  2 */ val y = 6
/*  3 */ def f(x: Int, y: Int) {
/*  4 */     x match {
/*  5 */         case z if x > 0 =>
/*  6 */             f(x - y, y)
/*  7 */         case _ =>
/*  8 */             ()
/*  9 */     }
/* 10 */ }
/* 11 */ f(y, x)

x: Int = 3
y: Int = 6
defined function f

1) {} <br>
2) {x: 3}<br>
3) {x: 3, y: 6}<br>
11) {x: 3, y: 6, f: func}<br>
4) {x: 6, y: 3, f: func}<br>
5) {x: 6, y: 3, f: func}<br>
6) {x: 6, y: 3, f: func, z: 6}<br>
4) {x: 3, y: 3, f: func}<br>
5) {x: 3, y: 3, f: func}<br>
6) {x: 3, y: 3, f: func, z: 3}<br>
4) {x: 0, y: 3, f: func}<br>
7) {x: 0, y: 3, f: func}<br>